## Exogenous Processes
This notebook contains prototypes for the implementation of exogenous processes.
I think the easiest is to start implementing the processes form a new branch that departs form current main. 
The only part that we can take over one to one is the model specification and parsing. 
The rest serves as good inspiration but has to be adapted substantially due to the new state space structure. 

In [2]:
import numpy as np
from functools import wraps
from scipy import special

import respy as rp

from respy.config import COVARIATES_DOT_PRODUCT_DTYPE
from respy.parallelization import parallelize_across_dense_dimensions
from respy.shared import create_dense_state_space_columns
from respy.shared import pandas_dot
from respy.solve import _create_choice_rewards
from respy.load_states import pandas_dot
from respy.pre_processing.model_processing import process_params_and_options

## Model Processing
- I would just take over the model processing from the old PR
- We have to copy all the functions that deal with exog processes to the new branch
- That is probably the last error prone way and it allows us to spot potential improvements

In [18]:
# Load model.
params, options = rp.get_example_model("robinson_crusoe_extended", with_data=False)

# Extend with observable characteristic.
params.loc[("observable_health_well", "probability"), "value"] = 0.9
params.loc[("observable_health_sick", "probability"), "value"] = 0.1
params.loc[("observable_ability_good", "probability"), "value"] = 0.9
params.loc[("observable_ability_bad", "probability"), "value"] = 0.1
params.loc[("observable_ability_horrible", "probability"), "value"] = 0.1



# Create internal specification objects.
optim_paras, options = process_params_and_options(params, options)

/home/mo2561057/OpenSourceEconomics/respy/respy/pre_processing/model_processing.py:590: UserWarning: The probabilities for parameter group \bobservable_ability_([0-9a-z_]+)\b do not sum to one.
  category=UserWarning,


In [24]:
optim_paras["observables"].keys()

dict_keys(['ability', 'health'])

In [19]:
# Add exog processes
sp = rp.state_space.create_state_space_class(optim_paras, options)

In [22]:
sp.dense_covariates_to_dense_index


DictType[UniTuple(int64 x 2),int64]<iv=None>({(0, 0): 0, (0, 1): 1, (1, 0): 2, (1, 1): 3, (2, 0): 4, (2, 1): 5})

## Implementation
The old implementation seems to calculate transition probabilities once while retrieving continuation values. 
This approach seems impractical now since we do not keep the full representation of dense_period_choice cores in working memory.
Furthermore I hold the opinion that it should be treated similair to wages or nonpecs to ensure readability, coherence and flexibility. 
There are essentially three different avenues we could take now:
- We can treat the transition matrix entirely like wages and nonpecuniary rewards. The upside is that this probably
  integrated into the code quite smoothly and that it requires the least calculations. The downside is this would be quite working
  memory intensive.

- We can treat transition probabilities like states. Upsides are reduced working memory usage and smooth integration. 
  The downside are quite some IO calls.
  
- We can calculate transition probabilities each period in the backwardinduction loop. Altough this does not suffer from the issues 
  above I consider this to be the messiest solution. It would also cast some issues for the simulation. 


For now I follow the first approach but I think once we agree on the general strcuture the difference should not be too large! 
Th Code cells that follow contain all additions and changes to the state space and solve modules.
Code is not tested yet! Some small issues still have to be fixed!


Imortant general features to ensure no matter what option we choose: 
-  We need to be certain that the dense grid always has fixed dense vars in the leading positions and exog processes in the last!
   dense_covraite = (observable_position, process_position)

In [ ]:
# Option 1 we build a df for all potential locations on the dense grid
def compute_transition_probabilities(states,
                                     core_key,
                                     complex_,
                                     dense_covariates_to_dense_index,
                                     dense_index_and_core_key_to_dense_key,
                                     optim_paras,
                                     options
                                     ):
    
    exogenous_processes = optim_paras["exogenous_processes"]
    
    # How does the accounting work here again? Would that actually work?
    static_dense_columns = optim_paras["observables"] # We also still need to add types. Rethink parsing to an extent?
    
    static_dense = list(states.loc[0,static_dense].values())
    
    dense_columns = create_dense_state_space_columns(optim_paras)
    
    levels_of_processes = [range(len(i)) for i in optim_paras["observables"].values()]
    comb_exog_procs = itertools.product(*levels_of_processes)
    
    # Needs to be created in here since that is dense-period-choice-core specific. 
    dense_index_to_exogenous = {dense_covariates_to_dense_index[(*static_dense, *exog)]:exog for exog in comb_exog_procs}
    dense_key_to_exogenous = {dense_index_and_core_key_to_dense_key[(core_key,key)]:vaue for key,value in dense_index_to_exogenous.items()}
    
    # Compute the probabilities for every exogenous process.
    probabilities = []
    for exog_proc in exogenous_processes:

        # Create the dot product of covariates and parameters.
        x_betas = []
        for params in exogenous_processes[exog_proc].values():
            x_beta = pandas_dot(states[params.index],params)
            x_betas.append(x_beta)

        probs = special.softmax(np.column_stack(x_betas), axis=1)
        probabilities.append(probs)
    
    # Prepare full Dataframe. If issues arrise we might want to switch typed dicts 
    df = pd.Dataframe(index=states.index)
    for dense in dense_index_to_exogenous:
        array = np.product.reduce(probs[proc][:,val] for proc,val in enumerate(dense_key_to_exogenous[dense]))
        df[dense] = array
    
    # We can maybe  directly dump that dataset?
    # I think we still have to discuss whether we realy want that!
    dump_container(df,"transition",complex_, options)
    

    return df

In [ ]:
# Gefällt mir irgendwie doch nciht so gut. Wäre cool wenn wir hier noch flexibler werden könnten. 
# Auch bezüglich der meisten 
@parallelize_across_dense_dimensions
def _create_param_specific_objects(    
    complex_,
    core_key,
    choice_set,
    dense_covariates_to_dense_index,
    dense_index_and_core_key_to_dense_key,
    optim_paras,
    options):
    """ Insert Docstring """
    states = load_states(complex_, options)
    wages, nonpecs = _create_choice_rewards(complex_, choice_set, optim_paras, options)
    if "exogenous_processes" in optim_paras:
        transition = compute_transition_probabilities(states,
                                        core_key,
                                         complex_,
                                         dense_covariates_to_dense_index,
                                         dense_index_and_core_key_to_dense_key,
                                         optim_paras,
                                         options)
    return wages, nonpecs, transition
    

In [ ]:
def solve(params, options, state_space):
    """Solve the model."""
    optim_paras, options = process_params_and_options(params, options)
        
    # TODO: Think how this could be used to handle fixed and floating variables more efficiently. 
    # Thinking about Polymorphisms etc. 
    wages, nonpecs, transition = _create_param_specific_objects(
        state_space.dense_key_to_complex,
        state_space.dense_key_to_core_key,
        state_space.dense_key_to_choice_set,
        state_space.dense_covariates_to_dense_index,
        state_space.dense_index_and_core_key_to_dense_key,
        optim_paras,
        options,
    )

    state_space.wages = wages
    state_space.nonpecs = nonpecs
    state_space.transition = transition

    state_space = _solve_with_backward_induction(state_space, optim_paras, options)

    return state_space


# We need a way to efficiently combine values!
I would propose to use another decorator to do the weighting. I think that avoids a large mess in the solve module and 
it does justice to the abstract extension of the model. 

In [ ]:
# TODO: Think of a less specific name. Something like 
def weight_dense_cores(func):
    """Wrapper around get continuation values"""
    @wraps(func)
    def decorator_weight_dense_cores(*args, transition, **kwargs):
        exogenous = True if transition is not None
        continuation_values = func(*args, **kwargs) 
        if is exogenous:
            weighted_continuation_values = continuation_values.copy() # Not sure whether this is a nice solution 
            for dense_key, transition_df in transition.items():
                weighted_columns = \
                [transition_df[ftr_key].values.reshape(len(transition_df),1)*continuation_values[ftr_key] for ftr_key in transition_df.columns]
                weighted_continuation_values[dense_key] = np.sum.reduce(weighted_columns)
            return weighted_continuation_values
        else:
            return continuation values
    return wrap_continuation_values

In [ ]:
    def get_continuation_values(self, period, transition):
        """Get continuation values.

        The function takes the expected value functions from the previous periods and
        then uses the indices of child states to put these expected value functions in
        the correct format. If period is equal to self.n_periods - 1 the function
        returns arrays of zeros si        self.expected_value_functions = Dict.empty(
            key_type=nb.types.int64, value_type=nb.types.float64[:]
        )
        for index, indices in self.dense_key_to_core_indices.items():
            self.expected_value_functions[index] = np.zeros(len(indices))nce we are in terminal states. Otherwise we retrieve
        expected value functions for next period and call
        :func:`_get_continuation_values` to assign continuation values to all choices
        within a period. (The object `subset_expected_value_functions` is required
        because we need a Numba typed dict but the function
        :meth:`StateSpace.get_attribute_from_period` just returns a normal dict)

        Returns
        -------
        continuation_values : numba.typed.Dict
            The continuation values for each dense key in a :class:`numpy.ndarray`.

        See also
        --------
        _get_continuation_values
            A more theoretical explanation can be found here: See :ref:`get continuation
            values <get_continuation_values>`.

        """
        if period == self.n_periods - 1:
            shapes = self.get_attribute_from_period("base_draws_sol", period)
            states = self.get_attribute_from_period("dense_key_to_core_indices", period)
            continuation_values = {
                key: np.zeros((states[key].shape[0], shapes[key].shape[1]))
                for key in shapes
            }
        else:
            child_indices = self.get_attribute_from_period("child_indices", period)
            expected_value_functions = self.get_attribute_from_period(
                "expected_value_functions", period + 1
            )
            subset_expected_value_functions = Dict.empty(
                key_type=nb.types.int64, value_type=nb.types.float64[:]
            )
            for key, value in expected_value_functions.items():
                subset_expected_value_functions[key] = value

            continuation_values = _get_continuation_values(
                self.get_attribute_from_period("dense_key_to_core_indices", period),
                self.get_attribute_from_period("dense_key_to_complex", period),
                child_indices,
                self.core_key_and_dense_index_to_dense_key,
                transition,
                bypass={"expected_value_functions": subset_expected_value_functions},
            )
        return continuation_values


In [25]:
@weight_dense_cores
@parallelize_across_dense_dimensions
@nb.njit
def _get_continuation_values(
    core_indices,
    dense_complex_index,
    child_indices,
    core_index_and_dense_vector_to_dense_index,
    expected_value_functions,
):
    """Get continuation values from child states.

    The continuation values are the discounted expected value functions from child
    states. This method allows to retrieve continuation values that were obtained in the
    model solution. In particular the function assigns continuation values to state
    choice combinations by using the child indices created in
    :func:`_collect_child_indices`.

    Returns
    -------
    continuation_values : numpy.ndarray
        Array with shape ``(n_states, n_choices)``. Maps core_key and choice into
        continuation value.

    """
    if len(dense_complex_index) == 3:
        period, choice_set, dense_idx = dense_complex_index
    elif len(dense_complex_index) == 2:
        period, choice_set = dense_complex_index
        dense_idx = 0

    n_choices = sum_over_numba_boolean_unituple(choice_set)

    n_states = core_indices.shape[0]

    continuation_values = np.zeros((len(core_indices), n_choices))
    for i in range(n_states):
        for j in range(n_choices):
            core_idx, row_idx = child_indices[i, j]
            idx = (core_idx, dense_idx)
            dense_choice = core_index_and_dense_vector_to_dense_index[idx]

            continuation_values[i, j] = expected_value_functions[dense_choice][row_idx]

    return continuation_values


24

## Simulation
The simulation constitutes a somewhat different issue in terms of splitting the model. 
I would propose to calculate transition probabilities in the _simulate_single_period step! 


In [53]:
@split_and_combine_df
@parallelize_across_dense_dimensions
def _simulate_single_period(
    df, choice_set, wages, nonpecs, continuation_values, optim_paras
):
    """Simulate individuals in a single period.

    The function performs the following sets:

    - Map individuals in one period to the states in the model.
    - Simulate choices and wages for those individuals.
    - Store additional information in a :class:`pandas.DataFrame` and return it.

    Until now this function assumes that there are no mixed constraints.
    See docs for more information!

    """
    valid_choices = select_valid_choices(optim_paras["choices"], choice_set)

    n_wages_raw = len(optim_paras["choices_w_wage"])
    n_wages = sum(choice_set[:n_wages_raw])

    # Get indices which connect states in the state space and simulated agents. Subtract
    # the minimum of indices (excluding invalid indices) because wages, etc. contain
    # only wages in this period and normal indices select rows from all wages.

    period_indices = df["core_index"].to_numpy()
    try:
        wages = wages[period_indices]
        nonpecs = nonpecs[period_indices]
        continuation_values = continuation_values[period_indices]
    except IndexError as e:
        raise Exception(
            "Simulated individuals could not be mapped to their corresponding states in"
            " the state space. This might be caused by a mismatch between "
            "option['core_state_space_filters'] and the initial conditions."
        ) from e

    draws_shock = df[[f"shock_reward_{c}" for c in valid_choices]].to_numpy()
    draws_shock_transformed = transform_base_draws_with_cholesky_factor(
        draws_shock, choice_set, optim_paras["shocks_cholesky"], optim_paras
    )

    draws_wage = df[[f"meas_error_wage_{c}" for c in valid_choices]].to_numpy()
    value_functions, flow_utilities = calculate_value_functions_and_flow_utilities(
        wages,
        nonpecs,
        continuation_values,
        draws_shock_transformed,
        optim_paras["beta_delta"],
    )
    choice = np.nanargmax(value_functions, axis=1)

    # Get choice replacement dict. There is too much positioning until now!
    wages = wages * draws_shock_transformed * draws_wage
    wages[:, n_wages:] = np.nan
    wage = np.choose(choice, wages.T)

    # We map choice positions to choice codes
    positions = [i for i, x in enumerate(optim_paras["choices"]) if x in valid_choices]
    for pos, val in enumerate(positions):
        choice = np.where(choice == pos, val, choice)

    # Store necessary information and information for debugging, etc..

    df["choice"] = choice
    df["wage"] = wage
    df["discount_rate"] = optim_paras["delta"]
    df["present_bias"] = optim_paras["beta"]

    for i, choice in enumerate(valid_choices):
        df[f"nonpecuniary_reward_{choice}"] = nonpecs[:, i]
        df[f"wage_{choice}"] = wages[:, i]
        df[f"flow_utility_{choice}"] = flow_utilities[:, i]
        df[f"value_function_{choice}"] = value_functions[:, i]
        df[f"continuation_value_{choice}"] = continuation_values[:, i]

    return df

(100,)